In [32]:
import pandas as pd
import numpy as np
import os

In [33]:
name = "A359_MSN216_S18LHS_SS"
folder_in = '01_Input/'
folder_out = '02_Data/'
folder_in = folder_in + name
folder_out = folder_out + name
folder_tables = folder_in + '/tables'

In [34]:
df_SEA2SE = pd.read_excel(folder_tables + '/SEA2SE_SS_MSN321.xlsx')
df_SEA = pd.read_excel(folder_tables + '/SEA_frame.xlsx')
df_SEFR = pd.read_excel(folder_tables + '/SE_frame.xlsx')
df_mp = pd.read_excel(folder_tables + '/metal_profile.xlsx')
df_cp = pd.read_excel(folder_tables + '/comp_profile.xlsx')
df_st = pd.read_excel(folder_tables + '/stacking.xlsx')
df_jt = pd.read_excel(folder_tables + '/joint_table.xlsx')
#df_fa = pd.read_excel(folder_tables + '/fastener_system.xlsx')
#df_stab = pd.read_excel(folder_tables + '/stabilo_table.xlsx')

In [35]:
df1 = df_SEA2SE.loc[df_SEA2SE['param'] == 'SE_Frame'].melt(id_vars=['P_pos', 'param'])
df1.rename(columns={'value': 'AIRBUSSE_FRAME', 'variable': 'Frame'}, inplace=True)
df2 = df_SEA2SE.loc[df_SEA2SE['param'] == 'SEA_Frame'].melt(id_vars=['P_pos', 'param'])
df2.rename(columns={'value': 'SEA', 'variable': 'Frame'}, inplace=True)
df3 = pd.merge(df2,df1,how='left',on=['Frame', 'P_pos'])
df3 = df3[pd.notnull(df3['SEA'])]
df3 = df3[['SEA', 'AIRBUSSE_FRAME']]
df_SEA2SE = df3.reset_index(drop=True)

In [36]:
df_SEA = df_SEA[pd.notnull(df_SEA['CLIPPROFILE'])]
df_SEA = df_SEA.drop(['INNERDOUBLERPROFILE', 'OUTERDOUBLERPROFILE'],axis=1)
df_SEA.head()

,AIRBUSSEA_FRAME,CLIPPROFILE,CLIPLENGTH (mm),CLIPPOSITIONX (mm),JOINT_FRA_CLIP,FRA_CLIP_HEADSIDE,JOINT_FRACLIP_SKIN,FRACLIP_SKIN_HEADSIDE,LOWERSTAB,LOWERSTABPOSITIONX (mm),UPPERSTAB,UPPERSTABPOSITIONX (mm)
32,FRA-Fra-FR072-ST008-009,Clip_C72_P8,208.67,0,FA_C72P_8,Frame,FP_C72P_8,Skin,Dummy Stabilo,208.67,Dummy Stabilo,0.0
33,FRA-Fra-FR072-ST009-010,Clip_C72_P9,220.00,0,FA_C72P_9,Frame,FP_C72P_9,Skin,Dummy Stabilo,220.00,Dummy Stabilo,0.0
34,FRA-Fra-FR072-ST010-011,Clip_C72_P10,208.67,0,FA_C72P_10,Frame,FP_C72P_10,Skin,Lower_Stabilo_C72P10,208.67,Dummy Stabilo,0.0
35,FRA-Fra-FR072-ST011-012,Clip_C72_P11,208.67,0,FA_C72P_11,Frame,FP_C72P_11,Skin,Lower_Stabilo_C72P11,208.67,Lower_Stabilo_C72P10,0.0
36,FRA-Fra-FR072-ST012-013,Clip_C72_P12,208.67,0,FA_C72P_12,Frame,FP_C72P_12,Skin,Lower_Stabilo_C72P12,208.67,Lower_Stabilo_C72P11,0.0


In [37]:
df = pd.merge(df_SEA,df_SEA2SE,how='left',left_on='AIRBUSSEA_FRAME',right_on='SEA').drop('SEA',axis=1)

In [38]:
df = pd.merge(df,df_SEFR[['AIRBUSSE_FRAME', 'PROFILE0', 'OFFSETY0 (mm)', 'OFFSETZ0 (mm)']], how='left', on='AIRBUSSE_FRAME' )
df.rename(columns={'PROFILE0': 'FRAME_PROFILE'}, inplace=True)

In [39]:
df.columns

Index(['AIRBUSSEA_FRAME', 'CLIPPROFILE', 'CLIPLENGTH (mm)',
       'CLIPPOSITIONX (mm)', 'JOINT_FRA_CLIP', 'FRA_CLIP_HEADSIDE',
       'JOINT_FRACLIP_SKIN', 'FRACLIP_SKIN_HEADSIDE', 'LOWERSTAB',
       'LOWERSTABPOSITIONX (mm)', 'UPPERSTAB', 'UPPERSTABPOSITIONX (mm)',
       'AIRBUSSE_FRAME', 'FRAME_PROFILE', 'OFFSETY0 (mm)', 'OFFSETZ0 (mm)'],
      dtype='object')

### Frame Metal profiles

In [40]:
df_mp2 = df_mp.iloc[:,np.r_[0:2,3,6,7,9,13,30, 51,52,56,62]].copy()
df_mp2['TA (mm)'] = df_mp2['TA (mm)'].fillna(df_mp2['TF (mm)'])
df_mp2['T (mm)'] = df_mp2['T (mm)'].fillna(df_mp2['TA (mm)'])
df_mp2.drop(['TA (mm)', 'TF (mm)'], axis=1, inplace=True)
df_mp2.fillna(0, inplace=True)
df_mp2.head()

,PROFILES NAME,MATERIAL TYPE,FAMILY,MATERIAL,BILLET (mm),BA (mm),BF (mm),H (mm),T (mm),TW (mm)
0,Clip_C85_P11,Metal,L,7010_T7451_Plate,105.0,105.0,0.0,98.00,0.0,3.0
1,Clip_C72_P24,Metal,T,7010_T7651_Plate,120.0,59.0,0.0,123.95,4.0,4.0
2,Clip_C73_P24,Metal,T,7010_T7651_Plate,120.0,59.0,0.0,123.95,4.0,4.0
3,Clip_C74_P24,Metal,T,7475_T7351_Plate,70.0,62.0,0.0,124.02,4.0,2.0
4,Clip_C75_P24,Metal,T,7475_T7351_Plate,70.0,62.0,0.0,124.02,4.0,2.0


In [41]:
df2 = pd.merge(df,df_mp2,how='left', left_on='FRAME_PROFILE',right_on='PROFILES NAME').drop('PROFILES NAME',axis=1)


In [42]:
df2.columns

Index(['AIRBUSSEA_FRAME', 'CLIPPROFILE', 'CLIPLENGTH (mm)',
       'CLIPPOSITIONX (mm)', 'JOINT_FRA_CLIP', 'FRA_CLIP_HEADSIDE',
       'JOINT_FRACLIP_SKIN', 'FRACLIP_SKIN_HEADSIDE', 'LOWERSTAB',
       'LOWERSTABPOSITIONX (mm)', 'UPPERSTAB', 'UPPERSTABPOSITIONX (mm)',
       'AIRBUSSE_FRAME', 'FRAME_PROFILE', 'OFFSETY0 (mm)', 'OFFSETZ0 (mm)',
       'MATERIAL TYPE', 'FAMILY', 'MATERIAL', 'BILLET (mm)', 'BA (mm)',
       'BF (mm)', 'H (mm)', 'T (mm)', 'TW (mm)'],
      dtype='object')

### Frame Compo Profiles

In [43]:
df_cp2 = df_cp.iloc[:,np.r_[0:2,3,6:9,14,19,24]].copy()
df_cp2.fillna(0, inplace=True)
df_cp2.head()

,PROFILES NAME,MATERIAL TYPE,FAMILY,FIBER1,FIBER2,FIBER3,BA (mm),BF (mm),H (mm)
0,Str-Min-1.104,Compo,Omega,S16SSLH-06F1,S16SSLH-06F1,S16SSLH-06F1,27.000,28.979882,30.278
1,Fra_C91P_7,Compo,Z,11707400,11707400,11707400,18.034,25.934000,80.010
2,Fra_C92P_7,Compo,Z,11705300,11705300,11705300,17.482,25.934000,80.000
3,Clip_C91_P7,Compo,L,11738014,0,0,26.360,0.000000,73.700
4,Clip_C92_P7,Compo,L,11738014,0,0,26.360,0.000000,73.700


In [44]:
df3 = pd.merge(df2,df_cp2,how='left', left_on='FRAME_PROFILE',right_on='PROFILES NAME').drop('PROFILES NAME',axis=1)
merge = ['MATERIAL TYPE', 'FAMILY', 'BA (mm)', 'BF (mm)', 'H (mm)']
for m in merge:
    df3[m] = df3[m+'_y'].fillna(df3[m+'_x'])
    df3.drop([m+'_x', m+'_y'], axis=1, inplace=True)

In [45]:
cols = ['MATERIAL', 'BILLET (mm)', 'T (mm)', 'TW (mm)', 'FIBER1', 'FIBER2',
        'FIBER3', 'MATERIAL TYPE', 'FAMILY', 'BA (mm)', 'BF (mm)', 'H (mm)']
cols1 = [col + "_frame" for col in cols]
cols2 = [col + "_clip" for col in cols]

df3.rename(columns = dict(zip(cols, cols1)), inplace=True)
df3.columns

Index(['AIRBUSSEA_FRAME', 'CLIPPROFILE', 'CLIPLENGTH (mm)',
       'CLIPPOSITIONX (mm)', 'JOINT_FRA_CLIP', 'FRA_CLIP_HEADSIDE',
       'JOINT_FRACLIP_SKIN', 'FRACLIP_SKIN_HEADSIDE', 'LOWERSTAB',
       'LOWERSTABPOSITIONX (mm)', 'UPPERSTAB', 'UPPERSTABPOSITIONX (mm)',
       'AIRBUSSE_FRAME', 'FRAME_PROFILE', 'OFFSETY0 (mm)', 'OFFSETZ0 (mm)',
       'MATERIAL_frame', 'BILLET (mm)_frame', 'T (mm)_frame', 'TW (mm)_frame',
       'FIBER1_frame', 'FIBER2_frame', 'FIBER3_frame', 'MATERIAL TYPE_frame',
       'FAMILY_frame', 'BA (mm)_frame', 'BF (mm)_frame', 'H (mm)_frame'],
      dtype='object')

### Clip profiles

In [46]:
df4 = pd.merge(df3,df_mp2,how='left', left_on='CLIPPROFILE',right_on='PROFILES NAME').drop('PROFILES NAME',axis=1)
df5 = pd.merge(df4,df_cp2,how='left', left_on='CLIPPROFILE',right_on='PROFILES NAME').drop('PROFILES NAME',axis=1)
merge = ['MATERIAL TYPE', 'FAMILY', 'BA (mm)', 'BF (mm)', 'H (mm)']
for m in merge:
    df5[m] = df5[m+'_y'].fillna(df5[m+'_x'])
    df5.drop([m+'_x', m+'_y'], axis=1, inplace=True)

df5.rename(columns = dict(zip(cols, cols2)), inplace=True)
df5.columns

Index(['AIRBUSSEA_FRAME', 'CLIPPROFILE', 'CLIPLENGTH (mm)',
       'CLIPPOSITIONX (mm)', 'JOINT_FRA_CLIP', 'FRA_CLIP_HEADSIDE',
       'JOINT_FRACLIP_SKIN', 'FRACLIP_SKIN_HEADSIDE', 'LOWERSTAB',
       'LOWERSTABPOSITIONX (mm)', 'UPPERSTAB', 'UPPERSTABPOSITIONX (mm)',
       'AIRBUSSE_FRAME', 'FRAME_PROFILE', 'OFFSETY0 (mm)', 'OFFSETZ0 (mm)',
       'MATERIAL_frame', 'BILLET (mm)_frame', 'T (mm)_frame', 'TW (mm)_frame',
       'FIBER1_frame', 'FIBER2_frame', 'FIBER3_frame', 'MATERIAL TYPE_frame',
       'FAMILY_frame', 'BA (mm)_frame', 'BF (mm)_frame', 'H (mm)_frame',
       'MATERIAL_clip', 'BILLET (mm)_clip', 'T (mm)_clip', 'TW (mm)_clip',
       'FIBER1_clip', 'FIBER2_clip', 'FIBER3_clip', 'MATERIAL TYPE_clip',
       'FAMILY_clip', 'BA (mm)_clip', 'BF (mm)_clip', 'H (mm)_clip'],
      dtype='object')

### Stacking

In [47]:
#df_st.iloc[1,3:].value_counts()
df_st2 = df_st.iloc[:,3:]
df_st2 = pd.concat([df_st,df_st2.apply(pd.value_counts, axis = 1)],axis=1)
df_st2.columns = df_st2.columns.map(str)
df_st2 = df_st2.drop(['-30.0', '-15.0', '30.0', '15.0'],axis=1)
df_st2['STACKING'] = df_st2['STACKING LABEL'].str.split('_').str[0]
df_st2 = (df_st2[['-45.0', '0.0','45.0', '90.0', 'STACKING']]).copy()

In [48]:
df6 = pd.merge(df5,df_st2,how='left',left_on='FIBER1_frame',right_on='STACKING').drop(['STACKING'],axis=1)
df7 = pd.merge(df6,df_st2,how='left',left_on='FIBER1_clip',right_on='STACKING', suffixes=['_frame','_clip']).drop(['STACKING'],axis=1)
stackings = ['-45.0_frame', '0.0_frame', '45.0_frame', '90.0_frame', '-45.0_clip',
             '0.0_clip', '45.0_clip', '90.0_clip']
df7[stackings] = df7[stackings].fillna(0)
df7['SEA_MSN216'] = df7['AIRBUSSEA_FRAME'].str.replace(r'-Fra','')
df7 = df7[pd.notnull(df7['FRAME_PROFILE'])]
df7.columns

Index(['AIRBUSSEA_FRAME', 'CLIPPROFILE', 'CLIPLENGTH (mm)',
       'CLIPPOSITIONX (mm)', 'JOINT_FRA_CLIP', 'FRA_CLIP_HEADSIDE',
       'JOINT_FRACLIP_SKIN', 'FRACLIP_SKIN_HEADSIDE', 'LOWERSTAB',
       'LOWERSTABPOSITIONX (mm)', 'UPPERSTAB', 'UPPERSTABPOSITIONX (mm)',
       'AIRBUSSE_FRAME', 'FRAME_PROFILE', 'OFFSETY0 (mm)', 'OFFSETZ0 (mm)',
       'MATERIAL_frame', 'BILLET (mm)_frame', 'T (mm)_frame', 'TW (mm)_frame',
       'FIBER1_frame', 'FIBER2_frame', 'FIBER3_frame', 'MATERIAL TYPE_frame',
       'FAMILY_frame', 'BA (mm)_frame', 'BF (mm)_frame', 'H (mm)_frame',
       'MATERIAL_clip', 'BILLET (mm)_clip', 'T (mm)_clip', 'TW (mm)_clip',
       'FIBER1_clip', 'FIBER2_clip', 'FIBER3_clip', 'MATERIAL TYPE_clip',
       'FAMILY_clip', 'BA (mm)_clip', 'BF (mm)_clip', 'H (mm)_clip',
       '-45.0_frame', '0.0_frame', '45.0_frame', '90.0_frame', '-45.0_clip',
       '0.0_clip', '45.0_clip', '90.0_clip', 'SEA_MSN216'],
      dtype='object')

In [ ]:
#fill na
fills = ['T (mm)_frame', 'TW (mm)_frame', 'T (mm)_clip', 'TW (mm)_clip']
df7[fills] = df7[fills].fillna(0)

In [49]:
if not os.path.exists(folder_out):
    os.mkdir(folder_out)
df7.to_pickle(folder_out+'/' +'df_'+name + '_geo.pkl')
df7.to_csv(folder_out+'/' +'df_'+name + '_geo.csv')